# Quantum Enhanced Auxiliary Field Quantum Monte Carlo

This work was done in collaboration with the Next Generation Computing team at [BASF](https://www.basf.com/global/en.html).

In this tutorial we implement a quantum-classical hybrid workflow for computing the ground state energies of a strongly interacting molecular system. The algorithm consists of two parts:


1. A variational quantum eigensolver that uses the quantum-number-preserving ansatz proposed by [Anselmetti et al. (2021)](https://doi.org/10.1088/1367-2630/ac2cb3) to generate a quantum trial wave function $|\Psi_T\rangle$ using CUDA Quantum.

2. An Auxiliary-Field Quantum Monte Carlo simulation that realizes a classical imaginary time evolution and collects the ground state energy estimates.


In [ ]:
# Package installs
!pip install pyscf==2.6.2
!pip install openfermion==1.6.1
!pip install ipie==0.7.1